# Coding: Machine Translation by RNN

 - Dataset: wmt-17, en-zh, select 5m high-quality pairs of data
 - Model: Seq2seq with Encoder & Decoder framework
 - GPU: 4090

# Seq2Seq Encoder-Decoder Architecture

## Overview

The Sequence-to-Sequence (Seq2Seq) model with Encoder-Decoder architecture is a neural network framework designed for tasks where both input and output are sequences of variable length, such as machine translation (English → Chinese in your case).

```
Input Sequence (English):  "Hello world"
                              ↓
                          [ENCODER]
                              ↓
                        Context Vector
                              ↓
                          [DECODER]
                              ↓
Output Sequence (Chinese): "你好世界"
```

## Architecture Components

### 1. Encoder

The encoder processes the input sequence and compresses the information into a fixed-size context vector (also called thought vector).

```
Input: [w1, w2, w3, ..., wn]
       ↓    ↓    ↓       ↓
    [RNN][RNN][RNN]...[RNN]
       ↓    ↓    ↓       ↓
    [h1] [h2] [h3] ... [hn] → Context Vector (hn)
```

#### Key Components:
- **Embedding Layer**: Converts input tokens to dense vectors
- **RNN Layers**: LSTM/GRU cells process the sequence sequentially
- **Hidden States**: Capture information at each time step
- **Final Context**: Last hidden state becomes the context vector

```python
# Pseudo-code structure
class Encoder:
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers)
    
    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        outputs, hidden = self.rnn(embedded)
        return hidden  # Context vectors
```

### 2. Decoder

The decoder generates the output sequence one token at a time, using the context vector from the encoder.

```
Context Vector (C) → [RNN] → [RNN] → [RNN] → ... → [RNN]
                      ↓       ↓       ↓             ↓
                    [y1]    [y2]    [y3]   ...   [yn]
```

#### Key Components:
- **Initial State**: Initialized with encoder's context vector
- **RNN Layers**: Generate hidden states for each output position
- **Output Projection**: Maps hidden states to vocabulary probabilities
- **Softmax**: Converts logits to probability distribution

```python
# Pseudo-code structure
class Decoder:
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers)
        self.output_projection = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, target_seq, encoder_hidden, encoder_cell):
        embedded = self.embedding(target_seq)
        outputs, _ = self.rnn(embedded, encoder_hidden)
        predictions = self.output_projection(outputs)
        return predictions
```

## Complete Architecture Flow

### Training Phase

```
1. Input Processing:
   English: "Hello world" → [101, 7592, 2088, 102] (tokenized)
   Chinese: "[BOS] 你好世界 [EOS]" → [101, 872, 1962, 686, 102] (tokenized)

2. Encoder Forward Pass:
   Input: [101, 7592, 2088, 102]
   ↓
   Embedding: [[0.1, 0.2, ...], [0.3, 0.4, ...], ...]
   ↓
   LSTM/GRU: h1, h2, h3, h4 → Context Vector (h4)

3. Decoder Forward Pass:
   Initial State: (h4) from encoder
   Input: [101, 872, 1962, 686]
   ↓
   LSTM/GRU: generates hidden states for each position
   ↓
   Output Projection: [vocab_size] logits for each position
   ↓
   Loss Calculation: CrossEntropy with targets [872, 1962, 686, 102]
```

### Inference Phase

```
1. Encode input sequence: "Hello world"
2. Initialize decoder with encoder's context vector
3. Start with [BOS] token
4. Generate tokens one by one:
   - Input: [BOS] → Output: 你 (probability distribution)
   - Input: [BOS] 你 → Output: 好
   - Input: [BOS] 你 好 → Output: 世
   - Input: [BOS] 你 好 世 → Output: 界
   - Input: [BOS] 你 好 世 界 → Output: [EOS] (stop)
```

## Mathematical Formulation

### Encoder
```
h_t = LSTM/GRU(embedding(x_t), h_{t-1})
context = h_n  # Final hidden state
```

### Decoder
```
s_t = LSTM/GRU(embedding(y_{t-1}), s_{t-1})  # s_0 = context
P(y_t | y_1...y_{t-1}, x) = softmax(W_s * s_t + b_s)
```

### Loss Function
```
Loss = -∑∑ log P(y_t^i | y_1^i...y_{t-1}^i, x^i)
```

## Architecture Advantages

1. **Variable Length Handling**: Can process sequences of different lengths
2. **End-to-End Learning**: Jointly optimizes encoder and decoder
3. **Context Preservation**: Encoder captures semantic meaning in context vector
4. **Language Agnostic**: Works for any language pair

## Architecture Limitations

1. **Information Bottleneck**: Fixed-size context vector may lose information
2. **Long Sequence Problem**: Difficulty with very long input sequences
3. **Sequential Processing**: Cannot parallelize during inference

## Improvements & Variants

1. **Attention Mechanism**: Allows decoder to focus on relevant encoder states
2. **Bidirectional Encoder**: Processes sequence in both directions
3. **Beam Search**: Better decoding strategy than greedy search
4. **Teacher Forcing**: Training technique using ground truth as decoder input

## Implementation Architecture for Your Project

Based on your dataset (WMT-17 EN-ZH) and tokenizers (BERT-based), here's the recommended architecture:

```
Input: English sentence (max_length=100)
↓
BERT Tokenizer (vocab_size=30522) → Token IDs
↓
Embedding Layer (30522 → 512)
↓
Encoder LSTM/GRU (512 → 1024, num_layers=2)
↓
Context Vector (1024-dim)
↓
Decoder LSTM/GRU (512 → 1024, num_layers=2)
↓
Output Projection (1024 → 21128)
↓
Chinese Token IDs → BERT Tokenizer → Chinese sentence
```

In [1]:
# Download the data & select 5m high-quality pairs

from datasets import load_dataset
import re

# load full wmt-17 en-zh dataset
full_dataset = load_dataset("wmt/wmt17", "zh-en", split="train")

# Length & Ratio filter
def is_high_quality(x):
    en = x["translation"]["en"]
    zh = x["translation"]["zh"]
    if not en or not zh:
        return False
    if len(en) < 3 or len(zh) < 3:
        return False
    if len(en) > 100 or len(zh) > 100:
        return False
    ratio = len(en) / len(zh)
    if ratio < 0.5 or ratio > 2:
        return False
    if not re.search(r'[\u4e00-\u9fff]', zh):
        return False
    return True

filtered_dataset = full_dataset.filter(is_high_quality, num_proc=10)

dataset = filtered_dataset.select(range(min(5_000_000, len(filtered_dataset))))

print("Full Dataset Size: ", len(full_dataset))
print("Filtered Dataset Size: ", len(filtered_dataset))
print("Dataset Size: ", len(dataset))

# print 10 samples
sample = dataset.shuffle(seed=42).select(range(10))
print("-"*100)
for i in sample:
    print(i["translation"]["en"])
    print(i["translation"]["zh"])
    print("-"*100)

/home/archer/archer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Full Dataset Size:  25134743
Filtered Dataset Size:  1141860
Dataset Size:  1141860
----------------------------------------------------------------------------------------------------
Zambia (7)
赞比亚(7)
----------------------------------------------------------------------------------------------------
15:00 to 18:00 Informal consultations (closed) Conference Room 5 (NLB)
下午3:00－6:00 非正式磋商(闭门会议) 第5会议室(北草坪会议大楼)
----------------------------------------------------------------------------------------------------
Spain
西班牙
----------------------------------------------------------------------------------------------------
Mr. Robert Morrison
Robert Morrison先生 加拿大自然资源部
----------------------------------------------------------------------------------------------------
This satisfied the kids, but not the husband.
"孩子们得到了满意的答案, 但她的丈夫却没有。
----------------------------------------------------------------------------------------------------
Shutaro Omura (Japan)
Shutaro Omura（日本）
---------------

In [2]:
# Create PyTorch Dataset and DataLoader for training

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

class TranslationDataset(Dataset):
    def __init__(self, hf_dataset, tokenizer_en, tokenizer_zh, max_length=100):
        """
        PyTorch Dataset wrapper for HuggingFace translation dataset
        
        Args:
            hf_dataset: HuggingFace dataset with translation pairs
            tokenizer_en: English tokenizer (optional, can be added later)
            tokenizer_zh: Chinese tokenizer (optional, can be added later)
            max_length: Maximum sequence length
        """
        self.dataset = hf_dataset
        self.tokenizer_en = tokenizer_en
        self.tokenizer_zh = tokenizer_zh
        self.max_length = max_length
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        item = self.dataset[idx]
        en_text = item["translation"]["en"]
        zh_text = item["translation"]["zh"]

        en_tokens = self.tokenizer_en(en_text, 
                                        max_length=self.max_length, 
                                        padding='max_length', 
                                        truncation=True, 
                                        return_tensors='pt')
            
        zh_tokens = self.tokenizer_zh(zh_text, 
                                        max_length=self.max_length, 
                                        padding='max_length', 
                                        truncation=True, 
                                        return_tensors='pt')
            
        return {
                'source_ids': en_tokens['input_ids'].squeeze(),
                'target_ids': zh_tokens['input_ids'].squeeze(),
                'source_text': en_text,
                'target_text': zh_text
        }

def create_dataloaders(dataset, batch_size=32, num_workers=4, train_split=0.95):
    """
    Create train and validation DataLoaders from HuggingFace dataset
    
    Args:
        dataset: HuggingFace dataset with translation pairs
        batch_size: Batch size for DataLoaders
        num_workers: Number of worker processes for data loading
        train_split: Fraction of data to use for training
    
    Returns:
        train_dataloader, val_dataloader, train_dataset, val_dataset
    """
    
    # Split dataset into train and validation
    train_size = int(train_split * len(dataset))
    
    # Create indices for splitting
    indices = list(range(len(dataset)))
    train_indices, val_indices = train_test_split(indices, 
                                                train_size=train_size, 
                                                random_state=42)
    
    # Create train and validation datasets
    train_dataset_hf = dataset.select(train_indices)
    val_dataset_hf = dataset.select(val_indices)

    # tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    tokenizer_en = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokenizer_zh = AutoTokenizer.from_pretrained("bert-base-chinese")

    # get vocab sizes
    vocab_size_en = tokenizer_en.vocab_size
    vocab_size_zh = tokenizer_zh.vocab_size

    print(f"Vocab size for en: {vocab_size_en}")
    print(f"Vocab size for zh: {vocab_size_zh}")

    # Create PyTorch datasets
    train_dataset = TranslationDataset(train_dataset_hf, tokenizer_en, tokenizer_zh)
    val_dataset = TranslationDataset(val_dataset_hf, tokenizer_en, tokenizer_zh)
    
    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    
    # Create DataLoaders
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    print(f"Train DataLoader: {len(train_dataloader)} batches")
    print(f"Validation DataLoader: {len(val_dataloader)} batches")
    
    return train_dataloader, val_dataloader, vocab_size_en, vocab_size_zh

def test_dataloader(dataloader):
    """Test the DataLoader by printing a sample batch"""
    print("\n" + "="*50)
    print("Sample batch from DataLoader:")
    print("="*50)
    
    for batch in dataloader:
        print(f"Batch size: {len(batch['source_text'])}")
        print(f"Source example: {batch['source_text'][0]}")
        print(f"Source tokens: {batch['source_ids'][0]}")
        print(f"Target example: {batch['target_text'][0]}")
        print(f"Target tokens: {batch['target_ids'][0]}")
        break

train_dataloader, val_dataloader, encoder_vocab_size, decoder_vocab_size = create_dataloaders(dataset)
test_dataloader(train_dataloader)
test_dataloader(val_dataloader)

Vocab size for en: 30522
Vocab size for zh: 21128
Train dataset size: 1084767
Validation dataset size: 57093
Train DataLoader: 33899 batches
Validation DataLoader: 1785 batches

Sample batch from DataLoader:
Batch size: 32
Source example: There were 132 cases of beatings.
Source tokens: tensor([  101,  2045,  2020, 14078,  3572,  1997,  6012,  2015,  1012,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
          

In [10]:
# Define the Seq2Seq model with GRU

import torch
import torch.nn as nn

class Encoder(nn.Module):
    """
    Encoder component of the Seq2Seq model using GRU
    Processes the input sequence and generates context vectors
    """
    def __init__(self, vocab_size, embed_size=512, hidden_size=1024, num_layers=2, dropout=0.1):
        super(Encoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Embedding layer to convert token IDs to dense vectors
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        
        # GRU layer for processing sequences
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers, 
                         batch_first=True, dropout=dropout, bidirectional=False)
        
    def forward(self, input_seq):
        """
        Forward pass of the encoder
        
        Args:
            input_seq: Input token sequences [batch_size, seq_len]
            input_lengths: Actual lengths of sequences (for packed sequences)
            
        Returns:
            outputs: All hidden states [batch_size, seq_len, hidden_size]
            hidden: Final hidden state [num_layers, batch_size, hidden_size]
        """
        
        # Convert token IDs to embeddings
        embedded = self.embedding(input_seq)  # [batch_size, seq_len, embed_size]
        
        # Pass through GRU
        outputs, hidden = self.rnn(embedded)
        
        # outputs: [batch_size, seq_len, hidden_size]
        # hidden: [num_layers, batch_size, hidden_size] 
        
        return outputs, hidden

class Decoder(nn.Module):
    """
    Decoder component of the Seq2Seq model using GRU
    Generates output sequence one token at a time
    """
    def __init__(self, vocab_size, embed_size=512, hidden_size=1024, num_layers=2, dropout=0.1):
        super(Decoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Embedding layer for target tokens
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        
        # GRU layer for generating sequences
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers, 
                         batch_first=True, dropout=dropout, bidirectional=False)
        
        # Output projection layer to vocabulary
        self.output_projection = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, input_token, hidden):
        """
        Forward pass of the decoder (single step)
        
        Args:
            input_token: Current input token [batch_size, 1]
            hidden: Hidden state from encoder/previous step [num_layers, batch_size, hidden_size]
            
        Returns:
            output: Vocabulary predictions [batch_size, vocab_size]
            hidden: Updated hidden state [num_layers, batch_size, hidden_size]
        """
        # input_token: [batch_size, 1]
        embedded = self.embedding(input_token)  # [batch_size, 1, embed_size]
        
        # Pass through GRU
        gru_out, hidden = self.rnn(embedded, hidden)
        # gru_out: [batch_size, 1, hidden_size]
        
        # Project to vocabulary
        output = self.output_projection(gru_out.squeeze(1))  # [batch_size, vocab_size]
        
        return output, hidden

class Seq2Seq(nn.Module):
    """
    Complete Sequence-to-Sequence model using GRU
    Combines Encoder and Decoder for translation
    """
    def __init__(self, encoder_vocab_size, decoder_vocab_size, embed_size=512, 
                 hidden_size=1024, num_layers=2, dropout=0.1):
        super(Seq2Seq, self).__init__()
        
        self.encoder_vocab_size = encoder_vocab_size
        self.decoder_vocab_size = decoder_vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Initialize encoder and decoder
        self.encoder = Encoder(encoder_vocab_size, embed_size, hidden_size, num_layers, dropout)
        self.decoder = Decoder(decoder_vocab_size, embed_size, hidden_size, num_layers, dropout)
        
    def forward(self, source_seq, target_seq):
        """
        Forward pass of the complete Seq2Seq model
        
        Args:
            source_seq: Source language sequences [batch_size, source_len]
            target_seq: Target language sequences [batch_size, target_len]
            teacher_forcing_ratio: Probability of using teacher forcing
            
        Returns:
            outputs: Predicted sequences [batch_size, target_len, vocab_size]
        """
        batch_size = source_seq.size(0)
        target_len = target_seq.size(1)
        
        # Encode the source sequence
        _, hidden = self.encoder(source_seq)
        
        # Store decoder outputs
        outputs = torch.zeros(batch_size, target_len, self.decoder_vocab_size).to(source_seq.device)
        
        # Decode step by step
        for t in range(target_len):
            # Get decoder output for current step
            decoder_input = target_seq[:, t].unsqueeze(1)
            output, hidden = self.decoder(decoder_input, hidden)
            outputs[:, t, :] = output
        
        return outputs
    
    def generate(self, source_seq, max_length=100, start_token=101, end_token=102):
        """
        Generate translation for given source sequence (inference mode)
        
        Args:
            source_seq: Source sequence [batch_size, source_len]
            max_length: Maximum length of generated sequence
            start_token: BOS token ID (101 for BERT)
            end_token: EOS token ID (102 for BERT)
            
        Returns:
            generated_seq: Generated sequence [batch_size, generated_len]
        """
        self.eval()
        batch_size = source_seq.size(0)
        
        with torch.no_grad():
            # Encode source sequence
            _, hidden = self.encoder(source_seq)
            
            # Initialize with start token
            decoder_input = torch.full((batch_size, 1), start_token, dtype=torch.long).to(source_seq.device)
            
            # Store generated tokens
            generated_tokens = []
            
            for _ in range(max_length):
                # Get next token prediction
                output, hidden = self.decoder(decoder_input, hidden)
                
                # Get the token with highest probability
                next_token = output.argmax(dim=1).unsqueeze(1)
                generated_tokens.append(next_token)
                
                # Use predicted token as next input
                decoder_input = next_token
                
                # Stop if all sequences generated EOS token
                if torch.all(next_token.squeeze() == end_token):
                    break
            
            # Concatenate all generated tokens
            generated_seq = torch.cat(generated_tokens, dim=1)
            
        return generated_seq

# Model configuration based on your dataset
model_config = {
    'encoder_vocab_size': encoder_vocab_size,  # 30522 (English BERT)
    'decoder_vocab_size': decoder_vocab_size,  # 21128 (Chinese BERT)
    'embed_size': 256,
    'hidden_size': 512,
    'num_layers': 2,
    'dropout': 0.1
}

# Initialize the model
model = Seq2Seq(**model_config)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print("=== Seq2Seq Model with GRU Architecture ===")
print(f"Device: {device}")
print(f"Encoder Vocabulary Size: {model_config['encoder_vocab_size']:,}")
print(f"Decoder Vocabulary Size: {model_config['decoder_vocab_size']:,}")
print(f"Embedding Size: {model_config['embed_size']}")
print(f"Hidden Size: {model_config['hidden_size']}")
print(f"Number of Layers: {model_config['num_layers']}")
print(f"Dropout Rate: {model_config['dropout']}")
print(f"RNN Type: GRU (Gated Recurrent Unit)")

# Calculate total parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Model Size: {total_params * 4 / 1024 / 1024:.2f} MB")

# Print model architecture
print(f"\n=== Model Architecture ===")
print(model)

# Test the model with a sample batch
print(f"\n=== Testing Model Forward Pass ===")
sample_batch = next(iter(train_dataloader))
source_ids = sample_batch['source_ids'].to(device)
target_ids = sample_batch['target_ids'].to(device)

print(f"Source shape: {source_ids.shape}")
print(f"Target shape: {target_ids.shape}")

# Forward pass
model.train()
outputs = model(source_ids, target_ids)
print(f"Output shape: {outputs.shape}")
print(f"Expected shape: [batch_size, target_len, decoder_vocab_size]")
print(f"Actual shape: [{outputs.shape[0]}, {outputs.shape[1]}, {outputs.shape[2]}]")

# Test generation
print(f"\n=== Testing Model Generation ===")
model.eval()
with torch.no_grad():
    generated = model.generate(source_ids[:2], max_length=100)  # Generate for first 2 samples
    print(f"Generated sequence shape: {generated.shape}")
    print(f"Generated tokens (first sample): {generated[0].tolist()}")

=== Seq2Seq Model with GRU Architecture ===
Device: cpu
Encoder Vocabulary Size: 30,522
Decoder Vocabulary Size: 21,128
Embedding Size: 256
Hidden Size: 512
Number of Layers: 2
Dropout Rate: 0.1
RNN Type: GRU (Gated Recurrent Unit)

Total Parameters: 29,578,376
Trainable Parameters: 29,578,376
Model Size: 112.83 MB

=== Model Architecture ===
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(30522, 256, padding_idx=0)
    (rnn): GRU(256, 512, num_layers=2, batch_first=True, dropout=0.1)
  )
  (decoder): Decoder(
    (embedding): Embedding(21128, 256, padding_idx=0)
    (rnn): GRU(256, 512, num_layers=2, batch_first=True, dropout=0.1)
    (output_projection): Linear(in_features=512, out_features=21128, bias=True)
  )
)

=== Testing Model Forward Pass ===
Source shape: torch.Size([32, 100])
Target shape: torch.Size([32, 100])
Output shape: torch.Size([32, 100, 21128])
Expected shape: [batch_size, target_len, decoder_vocab_size]
Actual shape: [32, 100, 21128]

=== Testing Model Ge

In [11]:
# Train the model with comprehensive training loop

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from collections import defaultdict

class Trainer:
    """
    Trainer class for Seq2Seq machine translation model
    """
    def __init__(self, model, train_dataloader, val_dataloader, device, 
                 learning_rate=1e-3, weight_decay=1e-5):
        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.device = device
        
        # Loss function - ignore padding tokens (index 0)
        self.criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='mean')
        
        # Optimizer
        self.optimizer = optim.Adam(model.parameters(), 
                                  lr=learning_rate, 
                                  weight_decay=weight_decay)
        
        # Learning rate scheduler
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=2, verbose=True
        )
        
        # Training history
        self.history = defaultdict(list)
        
    def calculate_loss(self, outputs, targets, target_lengths=None):
        """Calculate loss for the batch"""
        # Reshape for loss calculation (use reshape instead of view for non-contiguous tensors)
        outputs_flat = outputs.reshape(-1, outputs.size(-1))
        targets_flat = targets.reshape(-1)
        
        # Calculate loss
        loss = self.criterion(outputs_flat, targets_flat)
        
        return loss
    
    def train_epoch(self):
        """Train for one epoch"""
        self.model.train()
        epoch_loss = 0.0
        num_batches = 0
        
        progress_bar = tqdm(self.train_dataloader, desc="Training", leave=False)
        
        for batch_idx, batch in enumerate(progress_bar):
            # Move batch to device
            source_ids = batch['source_ids'].to(self.device)
            target_ids = batch['target_ids'].to(self.device)
            
            # Zero gradients
            self.optimizer.zero_grad()
            
            # Forward pass with teacher forcing
            decoder_input = target_ids[:, :-1]  # Remove last token
            decoder_targets = target_ids[:, 1:]  # Remove first token (BOS)
            
            outputs = self.model(source_ids, decoder_input)
            
            # Calculate loss
            loss = self.calculate_loss(outputs, decoder_targets)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            # Update parameters
            self.optimizer.step()
            
            # Update metrics
            epoch_loss += loss.item()
            num_batches += 1
            
            # Update progress bar
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Avg Loss': f'{epoch_loss / num_batches:.4f}'
            })
            
            # Memory cleanup
            if batch_idx % 100 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        return epoch_loss / num_batches
    
    def validate_epoch(self):
        """Validate for one epoch"""
        self.model.eval()
        epoch_loss = 0.0
        num_batches = 0
        
        progress_bar = tqdm(self.val_dataloader, desc="Validation", leave=False)
        
        with torch.no_grad():
            for batch in progress_bar:
                # Move batch to device
                source_ids = batch['source_ids'].to(self.device)
                target_ids = batch['target_ids'].to(self.device)
                
                # Forward pass
                decoder_input = target_ids[:, :-1]
                decoder_targets = target_ids[:, 1:]
                
                outputs = self.model(source_ids, decoder_input)
                
                # Calculate loss
                loss = self.calculate_loss(outputs, decoder_targets)
                
                # Update metrics
                epoch_loss += loss.item()
                num_batches += 1
                
                # Update progress bar
                progress_bar.set_postfix({
                    'Val Loss': f'{loss.item():.4f}',
                    'Avg Val Loss': f'{epoch_loss / num_batches:.4f}'
                })
        
        return epoch_loss / num_batches
    
    def train(self, num_epochs=10, save_path=None):
        """Train the model for specified number of epochs"""
        print(f"=== Starting Training for {num_epochs} Epochs ===")
        print(f"Device: {self.device}")
        print(f"Train batches: {len(self.train_dataloader)}")
        print(f"Validation batches: {len(self.val_dataloader)}")
        print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        print("=" * 60)
        
        best_val_loss = float('inf')
        start_time = time.time()
        
        for epoch in range(num_epochs):
            epoch_start_time = time.time()
            
            print(f"\nEpoch {epoch + 1}/{num_epochs}")
            print("-" * 40)
            
            # Train
            train_loss = self.train_epoch()
            
            # Validate
            val_loss = self.validate_epoch()
            
            # Update learning rate scheduler
            self.scheduler.step(val_loss)
            
            # Calculate epoch time
            epoch_time = time.time() - epoch_start_time
            
            # Store history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['epoch_time'].append(epoch_time)
            self.history['learning_rate'].append(self.optimizer.param_groups[0]['lr'])
            
            # Print epoch summary
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Val Loss: {val_loss:.4f}")
            print(f"Learning Rate: {self.optimizer.param_groups[0]['lr']:.6f}")
            print(f"Epoch Time: {epoch_time:.2f}s")
            
            # Save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print(f"New best validation loss: {best_val_loss:.4f}")
                
                if save_path:
                    torch.save({
                        'epoch': epoch + 1,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'train_loss': train_loss,
                        'val_loss': val_loss,
                        'model_config': model_config
                    }, save_path)
                    print(f"Model saved to {save_path}")
            
            # Memory cleanup
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        # Training summary
        total_time = time.time() - start_time
        print(f"\n{'='*60}")
        print(f"Training completed in {total_time:.2f}s ({total_time/60:.2f} min)")
        print(f"Best validation loss: {best_val_loss:.4f}")
        print(f"Final train loss: {self.history['train_loss'][-1]:.4f}")
        print(f"Final val loss: {self.history['val_loss'][-1]:.4f}")
        
        return self.history
    
    def plot_training_history(self):
        """Plot training history"""
        if not self.history['train_loss']:
            print("No training history to plot")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot loss
        axes[0, 0].plot(self.history['train_loss'], label='Train Loss', color='blue')
        axes[0, 0].plot(self.history['val_loss'], label='Validation Loss', color='red')
        axes[0, 0].set_title('Training and Validation Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
        
        # Plot learning rate
        axes[0, 1].plot(self.history['learning_rate'], color='green')
        axes[0, 1].set_title('Learning Rate Schedule')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Learning Rate')
        axes[0, 1].grid(True)
        
        # Plot epoch time
        axes[1, 0].plot(self.history['epoch_time'], color='orange')
        axes[1, 0].set_title('Epoch Training Time')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Time (seconds)')
        axes[1, 0].grid(True)
        
        # Plot loss comparison
        x = range(len(self.history['train_loss']))
        axes[1, 1].fill_between(x, self.history['train_loss'], alpha=0.3, color='blue', label='Train Loss')
        axes[1, 1].fill_between(x, self.history['val_loss'], alpha=0.3, color='red', label='Val Loss')
        axes[1, 1].plot(self.history['train_loss'], color='blue', linewidth=2)
        axes[1, 1].plot(self.history['val_loss'], color='red', linewidth=2)
        axes[1, 1].set_title('Loss Comparison')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Loss')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
        
        plt.tight_layout()
        plt.show()

# Initialize trainer
trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    learning_rate=1e-3,
    weight_decay=1e-5
)

# Start training for 10 epochs
print("Starting training...")
history = trainer.train(num_epochs=10, save_path="best_seq2seq_model.pth")

# Plot training history
print("\nPlotting training history...")
trainer.plot_training_history()


Starting training...
=== Starting Training for 10 Epochs ===
Device: cpu
Train batches: 33899
Validation batches: 1785
Model parameters: 29,578,376

Epoch 1/10
----------------------------------------


Training:   0%|          | 0/33899 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Comprehensive Machine Translation Testing Code

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer
import numpy as np
from collections import Counter
import re
import time
import math

class TranslationTester:
    """
    Comprehensive tester for the Seq2Seq machine translation model
    """
    def __init__(self, model, device, max_length=100):
        self.model = model
        self.device = device
        self.max_length = max_length
        
        # Load tokenizers
        self.tokenizer_en = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.tokenizer_zh = AutoTokenizer.from_pretrained("bert-base-chinese")
        
        # Special tokens
        self.start_token = 101  # [BOS] token
        self.end_token = 102   # [EOS] token
        self.pad_token = 0     # [PAD] token
        
        print("=== Translation Tester Initialized ===")
        print(f"Device: {device}")
        print(f"English tokenizer vocab size: {self.tokenizer_en.vocab_size}")
        print(f"Chinese tokenizer vocab size: {self.tokenizer_zh.vocab_size}")
        print(f"Max sequence length: {max_length}")

    def preprocess_text(self, text):
        """Clean and preprocess input text"""
        # Basic cleaning
        text = text.strip()
        text = re.sub(r'\s+', ' ', text)  # Multiple spaces to single space
        return text

    def encode_english(self, text):
        """Encode English text to token IDs"""
        text = self.preprocess_text(text)
        tokens = self.tokenizer_en(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return tokens['input_ids'].to(self.device)

    def decode_chinese(self, token_ids):
        """Decode Chinese token IDs to text"""
        # Remove padding tokens and special tokens
        if isinstance(token_ids, torch.Tensor):
            token_ids = token_ids.cpu().tolist()
        
        # Remove padding (0), start (101), and end (102) tokens
        cleaned_tokens = []
        for token_id in token_ids:
            if token_id not in [0, 101, 102]:
                cleaned_tokens.append(token_id)
            elif token_id == 102:  # Stop at end token
                break
        
        if not cleaned_tokens:
            return "[Empty translation]"
        
        try:
            text = self.tokenizer_zh.decode(cleaned_tokens, skip_special_tokens=True)
            # Clean up extra spaces
            text = re.sub(r'\s+', '', text)  # Remove all spaces for Chinese
            return text if text else "[Empty translation]"
        except Exception as e:
            return f"[Decoding error: {str(e)}]"

    def translate_single(self, english_text, beam_size=1, temperature=1.0):
        """
        Translate a single English sentence to Chinese
        
        Args:
            english_text: Input English text
            beam_size: Beam search size (1 for greedy)
            temperature: Sampling temperature for generation
        """
        self.model.eval()
        
        with torch.no_grad():
            # Encode input
            source_ids = self.encode_english(english_text)
            
            if beam_size == 1:
                # Greedy decoding
                generated = self.model.generate(
                    source_ids, 
                    max_length=self.max_length,
                    start_token=self.start_token,
                    end_token=self.end_token
                )
                translation = self.decode_chinese(generated[0])
            else:
                # Beam search (simplified version)
                translation = self._beam_search_translate(source_ids, beam_size, temperature)
        
        return translation

    def _beam_search_translate(self, source_ids, beam_size=3, temperature=1.0):
        """Simplified beam search implementation"""
        batch_size = source_ids.size(0)
        
        # Encode source
        _, hidden = self.model.encoder(source_ids)
        
        # Initialize beams
        beams = [(torch.tensor([[self.start_token]]).to(self.device), 0.0, hidden)]
        
        for step in range(self.max_length):
            new_beams = []
            
            for seq, score, h in beams:
                if seq[0, -1].item() == self.end_token:
                    new_beams.append((seq, score, h))
                    continue
                
                # Get next token probabilities
                last_token = seq[:, -1:] 
                output, new_h = self.model.decoder(last_token, h)
                
                # Apply temperature
                if temperature != 1.0:
                    output = output / temperature
                
                probs = F.softmax(output, dim=-1)
                
                # Get top k candidates
                top_probs, top_indices = torch.topk(probs, beam_size)
                
                for i in range(beam_size):
                    token_id = top_indices[0, i].unsqueeze(0).unsqueeze(0)
                    token_prob = top_probs[0, i].item()
                    new_seq = torch.cat([seq, token_id], dim=1)
                    new_score = score + math.log(token_prob + 1e-10)
                    new_beams.append((new_seq, new_score, new_h))
            
            # Keep only top beams
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
            
            # Check if all beams ended
            if all(seq[0, -1].item() == self.end_token for seq, _, _ in beams):
                break
        
        # Return best beam
        best_seq = beams[0][0]
        return self.decode_chinese(best_seq[0])

    def calculate_bleu_score(self, reference, candidate):
        """
        Calculate BLEU score (simplified version)
        """
        def get_ngrams(tokens, n):
            return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
        
        # Tokenize (character-level for Chinese)
        ref_tokens = list(reference.replace(' ', ''))
        cand_tokens = list(candidate.replace(' ', ''))
        
        if len(cand_tokens) == 0:
            return 0.0
        
        # Calculate precision for n-grams (1 to 4)
        precisions = []
        for n in range(1, 5):
            ref_ngrams = Counter(get_ngrams(ref_tokens, n))
            cand_ngrams = Counter(get_ngrams(cand_tokens, n))
            
            if len(cand_ngrams) == 0:
                precisions.append(0.0)
                continue
            
            matches = sum((ref_ngrams & cand_ngrams).values())
            precision = matches / len(get_ngrams(cand_tokens, n))
            precisions.append(precision)
        
        # Calculate brevity penalty
        ref_len = len(ref_tokens)
        cand_len = len(cand_tokens)
        
        if cand_len > ref_len:
            bp = 1.0
        else:
            bp = math.exp(1 - ref_len / (cand_len + 1e-10))
        
        # Calculate BLEU score
        if min(precisions) == 0:
            return 0.0
        
        bleu = bp * math.exp(sum(math.log(p + 1e-10) for p in precisions) / 4)
        return bleu

    def test_examples(self, test_cases, beam_size=1, temperature=1.0):
        """
        Test the model on multiple examples
        """
        print("\n" + "="*80)
        print("MACHINE TRANSLATION TESTING RESULTS")
        print("="*80)
        print(f"Model: Seq2Seq with GRU")
        print(f"Beam size: {beam_size}")
        print(f"Temperature: {temperature}")
        print("-"*80)
        
        results = []
        total_time = 0
        
        for i, (english, expected_chinese) in enumerate(test_cases, 1):
            print(f"\nTest {i}/{len(test_cases)}")
            print("-" * 40)
            print(f"English:  {english}")
            
            # Translate
            start_time = time.time()
            translation = self.translate_single(english, beam_size, temperature)
            translation_time = time.time() - start_time
            total_time += translation_time
            
            print(f"Expected: {expected_chinese}")
            print(f"Generated: {translation}")
            print(f"Time: {translation_time:.3f}s")
            
            # Calculate BLEU score if reference is provided
            if expected_chinese and expected_chinese != "":
                bleu_score = self.calculate_bleu_score(expected_chinese, translation)
                print(f"BLEU Score: {bleu_score:.4f}")
            else:
                bleu_score = None
                print("BLEU Score: N/A (no reference)")
            
            results.append({
                'english': english,
                'expected': expected_chinese,
                'translation': translation,
                'time': translation_time,
                'bleu': bleu_score
            })
        
        # Summary statistics
        print("\n" + "="*80)
        print("SUMMARY STATISTICS")
        print("="*80)
        print(f"Total examples: {len(test_cases)}")
        print(f"Total time: {total_time:.3f}s")
        print(f"Average time per translation: {total_time/len(test_cases):.3f}s")
        
        bleu_scores = [r['bleu'] for r in results if r['bleu'] is not None]
        if bleu_scores:
            print(f"Average BLEU score: {np.mean(bleu_scores):.4f}")
            print(f"Max BLEU score: {max(bleu_scores):.4f}")
            print(f"Min BLEU score: {min(bleu_scores):.4f}")
        
        return results

    def interactive_translation(self):
        """Interactive translation mode"""
        print("\n" + "="*60)
        print("INTERACTIVE TRANSLATION MODE")
        print("="*60)
        print("Enter English text to translate (type 'quit' to exit)")
        print("-"*60)
        
        while True:
            try:
                english_text = input("\nEnglish: ").strip()
                
                if english_text.lower() in ['quit', 'exit', 'q']:
                    print("Goodbye!")
                    break
                
                if not english_text:
                    print("Please enter some text.")
                    continue
                
                # Translate
                start_time = time.time()
                translation = self.translate_single(english_text)
                translation_time = time.time() - start_time
                
                print(f"Chinese:  {translation}")
                print(f"Time: {translation_time:.3f}s")
                
            except KeyboardInterrupt:
                print("\nGoodbye!")
                break
            except Exception as e:
                print(f"Error: {str(e)}")

# Real-world test cases for English to Chinese translation
REAL_WORLD_TEST_CASES = [
    # Basic greetings and common phrases
    ("Hello, how are you?", "你好，你好吗？"),
    ("Good morning!", "早上好！"),
    ("Thank you very much.", "非常感谢。"),
    ("Nice to meet you.", "很高兴见到你。"),
    ("See you later.", "再见。"),
    
    # Simple sentences
    ("I love you.", "我爱你。"),
    ("What is your name?", "你叫什么名字？"),
    ("Where are you from?", "你来自哪里？"),
    ("How much does this cost?", "这个多少钱？"),
    ("Can you help me?", "你能帮助我吗？"),
    
    # Daily life
    ("I am hungry.", "我饿了。"),
    ("The weather is nice today.", "今天天气很好。"),
    ("I need to go to work.", "我需要去工作。"),
    ("Let's have dinner together.", "我们一起吃晚饭吧。"),
    ("I am learning Chinese.", "我在学中文。"),
    
    # Travel and directions
    ("Where is the hotel?", "酒店在哪里？"),
    ("I want to go to the airport.", "我想去机场。"),
    ("How do I get to the train station?", "我怎么去火车站？"),
    ("Is this the right way?", "这是正确的路吗？"),
    ("I am lost.", "我迷路了。"),
    
    # Business and technology
    ("I work in technology.", "我在科技行业工作。"),
    ("The meeting is at 3 PM.", "会议在下午3点。"),
    ("Please send me the document.", "请发给我文件。"),
    ("This is a good opportunity.", "这是一个好机会。"),
    ("We need to discuss this further.", "我们需要进一步讨论这个。"),
    
    # Complex sentences
    ("Although it was raining, we decided to go out.", "虽然下雨了，我们还是决定出去。"),
    ("The book that I read yesterday was very interesting.", "我昨天读的那本书很有趣。"),
    ("If you study hard, you will succeed.", "如果你努力学习，你会成功的。"),
    ("She said she would come, but she didn't show up.", "她说她会来，但她没有出现。"),
    ("The restaurant we went to last night had excellent food.", "我们昨晚去的餐厅食物很棒。"),
    
    # Without reference translations (for testing only)
    ("Machine learning is revolutionizing many industries.", ""),
    ("Climate change is one of the biggest challenges of our time.", ""),
    ("The internet has connected people around the world.", ""),
    ("Artificial intelligence will shape the future of humanity.", ""),
    ("Education is the key to unlocking human potential.", ""),
]

# Initialize the tester
print("Initializing translation tester...")
tester = TranslationTester(model, device, max_length=100)

# Test with real-world examples
print("\nTesting with real-world examples...")
results = tester.test_examples(REAL_WORLD_TEST_CASES[:10], beam_size=1, temperature=1.0)

# Test with beam search (more advanced decoding)
print("\n" + "="*80)
print("TESTING WITH BEAM SEARCH (Beam size = 3)")
print("="*80)
beam_results = tester.test_examples(REAL_WORLD_TEST_CASES[10:15], beam_size=3, temperature=0.8)

# Compare different decoding strategies
print("\n" + "="*80)
print("COMPARING DECODING STRATEGIES")
print("="*80)

test_sentence = "I love learning new languages."
expected = "我喜欢学习新语言。"

print(f"Test sentence: {test_sentence}")
print(f"Expected: {expected}")
print("-" * 60)

# Greedy decoding
greedy_translation = tester.translate_single(test_sentence, beam_size=1, temperature=1.0)
greedy_bleu = tester.calculate_bleu_score(expected, greedy_translation)
print(f"Greedy:       {greedy_translation} (BLEU: {greedy_bleu:.4f})")

# Beam search
beam_translation = tester.translate_single(test_sentence, beam_size=3, temperature=0.8)
beam_bleu = tester.calculate_bleu_score(expected, beam_translation)
print(f"Beam Search:  {beam_translation} (BLEU: {beam_bleu:.4f})")

# High temperature (more random)
random_translation = tester.translate_single(test_sentence, beam_size=1, temperature=1.5)
random_bleu = tester.calculate_bleu_score(expected, random_translation)
print(f"High Temp:    {random_translation} (BLEU: {random_bleu:.4f})")

# Performance analysis
print("\n" + "="*80)
print("PERFORMANCE ANALYSIS")
print("="*80)

# Test translation speed with different sentence lengths
speed_test_cases = [
    "Hello.",  # Short
    "How are you doing today?",  # Medium
    "I really enjoy reading books and learning about different cultures around the world.",  # Long
    "The quick brown fox jumps over the lazy dog while the sun is shining brightly in the clear blue sky."  # Very long
]

print("Testing translation speed with different sentence lengths:")
print("-" * 60)

for i, sentence in enumerate(speed_test_cases, 1):
    start_time = time.time()
    translation = tester.translate_single(sentence)
    end_time = time.time()
    
    print(f"Length {i} ({len(sentence.split())} words): {end_time - start_time:.3f}s")
    print(f"  Input:  {sentence}")
    print(f"  Output: {translation}")
    print()

# Start interactive mode (optional)
print("\n" + "="*80)
print("Would you like to try interactive translation? (y/n)")
response = input().strip().lower()

if response in ['y', 'yes']:
    tester.interactive_translation()
else:
    print("Testing completed! You can run `tester.interactive_translation()` anytime for manual testing.")